# 決定木分類の学習と可視化
ch14-01とch14-03の可視化を合わせて行っています。


## データの読み込み

In [0]:
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from IPython.display import Image
from sklearn import tree

# データをロード
iris = datasets.load_iris()
# 説明変数
features = iris.data
# 目的変数
target = iris.target

print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

## データの内容

In [0]:
from pandas import DataFrame

# 説明変数と目的変数を結合
df = DataFrame(features, columns=iris.feature_names)
df['target'] = iris.target_names[iris.target]
df

# petal:花弁 sepal:がく弁

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [0]:
print(iris.target)
print(iris.target_names)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
['setosa' 'versicolor' 'virginica']


## 決定木分類モデルによる学習
不純度指標をジニ係数として学習を行う

In [0]:
# 決定木分類器オブジェクトを作成(デフォルトでジニ不純度を不純度指標としている)
decisiontree = DecisionTreeClassifier(random_state=0)
# decisiontree = DecisionTreeClassifier(criterion="entropy", random_state=0)

# 分類器を訓練
model = decisiontree.fit(features, target)

# DOTフォーマットでデータを作成
dot_data = tree.export_graphviz(decisiontree,
                                out_file=None,
                                feature_names=iris.feature_names,
                                class_names=iris.target_names)

# グラフを描画
graph = pydotplus.graph_from_dot_data(dot_data)

# グラフを表示
Image(graph.create_png())

graph.write_pdf("iris.pdf")

True

## 決定木分類モデルによる分類

In [0]:
# 新しい観測値を作成
observation = [[6.2, 2.6, 5.1, 2.0]]

# 観測値のクラスを予測
iris.target_names[model.predict(observation)]

array(['virginica'], dtype='<U10')

# 決定木回帰の学習と可視化
ch14-02とch14-03の可視化を合わせて行っています。

## データの読み込み

In [0]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import datasets
import pydotplus
from IPython.display import Image
from sklearn import tree

# データをロードして、2つだけ特徴量を残す。
boston = datasets.load_boston()
# 説明変数(特徴量)
features = boston.data[:, 0:2] [行(:)→全ての行:, 列(0:2)→0~2列目まで]
# 目的変数
target = boston.target
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

### データの内容

| 列名 | 説明 |
|:----:|:----:|
| CRIM | 人口1人あたりの犯罪発生率 |
| ZN | 25,000平方フィート以上の居住区画の占める割合 |
|MEDV|住宅価格の中央値（単位 1000ドル）|

* CRIMとZNを基にMEDVを予測する。説明変数。  
* MEDVが予測する目的変数。

In [0]:
from pandas import DataFrame

# 説明変数と目的変数を結合 featuresをpandasでデータフレーム化して
df = DataFrame(features, columns = boston.feature_names[0:2])
df['MEDV'] = boston.target
df

,CRIM,ZN,MEDV
0,0.00632,18.0,24.0
1,0.02731,0.0,21.6
2,0.02729,0.0,34.7
3,0.03237,0.0,33.4
4,0.06905,0.0,36.2
...,...,...,...
501,0.06263,0.0,22.4
502,0.04527,0.0,20.6
503,0.06076,0.0,23.9
504,0.10959,0.0,22.0


## 決定木回帰モデルによる学習
不純度指標をMSE(平均二乗誤差)として学習を行う

In [0]:
# 決定木回帰器オブジェクトを作成(デフォルトでMSEを不純度指標としている)
decisiontree = DecisionTreeRegressor(random_state=0)
# decisiontree_mae = DecisionTreeRegressor(criterion="mae", random_state=0)

# 決定木回帰器を訓練
model = decisiontree.fit(features, target)
print(model)

# DOTフォーマットでデータを作成
dot_data = tree.export_graphviz(decisiontree,
                                out_file=None,
                                feature_names=boston.feature_names[0:2],
                                class_names='MEDV')

# グラフを描画
graph = pydotplus.graph_from_dot_data(dot_data)

# グラフを表示
Image(graph.create_png())

graph.write_pdf("iris.pdf")

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')


True

## 決定木回帰モデルによる予測

In [0]:
# 新しい観測値の作成
observation = [[0.06260, 0.0]]

# 観測値のターゲット値を予測
model.predict(observation)

array([22.4])